## Auxiliary steps and makefile-style workflows

Auxiliary steps are special steps that are executed to provide [targets](Understanding_Targets.html) that are required by others.

For example, when the following step is executed with an input file `bamfile` (with extension `.bam`), it checks the existence of input file (`bamfile`), and a dependent index file (with extension `.bam.bai`).

```sos
[100 (call variant)]
input:   bamfile
depends: bamfile + '.bai'
run:
    # commands to call variants from 
    # input bam file
```

If the index file exists, generated either by another step or outside of SoS, sos will go ahead and execute the step. Otherwise  SoS will look in the script for a step that provides such a target, which would be similar to 

```sos
[index_bam : provides='{sample}.bam.bai']
input: "${sample}.bam"
run:
     samtools index ${input}
```

Such a step is characterized by a **`provides`** option (or a **`shared`** option that will be discussed later) and is called an auxiliary step. In this particular case, if `bamfile="AS123.bam"`, the requested dependent file would be `AS123.bam.bai`. Through the matching mechanism of option `provides`, the `index_bam` step would be executed with variables `sample="AS123"` and `output=["AS123.bam.bai"]`.

An auxiliary step can trigger other auxiliary steps and form a DAG (Directed Acyclic Graph). Acutually, you can write workflows in a make-file style with all auxiliary steps and execute workflows defined by targets. If you are familiar with Makefile, especially [snakemake](https://bitbucket.org/johanneskoester/snakemake), it can be natural for you to implement your workflow in this style. The advantage of SoS is that **you can use either or both forward-style and makefile-style steps to define your workflow** and take advantages of both approaches. For example, people frequently need to create fake targets to trigger steps that do not produce any target in a makefile-style workflow system, but this is not needed in SoS because steps defined in forward-style will always be executed. 

## Step option `provides`

An auxiliary step can be defined in the format of

```sos
[step_name : provides=pattern]
```

where `pattern` can be

* A file pattern such as `"{sample}.bam.idx"`
* Other types of targets such as `executable("ms")`
* A list (sequence) of one or more file patterns and targets.

### File targets

A file pattern is a filename with optional patterns with variable names enbraced in `{ }`. SoS matches filenames with the patterns and, if successful, assign variables with matched parts of the names. 

The following example removes all local `*.bam` and `*.bam.bi` file before it executes three workflows defined by `targets`. We could execute them from command line using commands such as
```bash
    sos run myscript -t TS1.bam
```
    if the script is defined in `myscript.sos`, or from Jupyter notebook using `%run` magic
```bash
    %run -t TS1.bam
```
but using action `sos_run` allows us to execute multiple workflows as nested workflows.

In [1]:
%sandbox

[compress: provides = '{filename}.bam']
print("> ${step_name} input to ${output}")
sh:
    touch ${output}

[index: provides = '{filename}.bam.bai']
input: "${filename}.bam"
print("> ${step_name} ${input} to ${output}")
sh:
    touch ${output}

[default]
print('Generating target TS1.bam')
sos_run(targets='TS1.bam')

print('\nGenerating target TS1.bam.bai')
sos_run(targets='TS1.bam.bai')

print('\nGenerating target TS2.bam.bai')
sos_run(targets='TS2.bam.bai')


Generating target TS1.bam
> compress input to TS1.bam

Generating target TS1.bam.bai
> index TS1.bam to TS1.bam.bai

Generating target TS2.bam.bai
> compress input to TS2.bam
> index TS2.bam to TS2.bam.bai


## -- Preview output --

input: TS2.bam 
output: TS2.bam.bai

> TS2.bam.bai (0 B):

As you can see from the output, when the first workflow is executed with target `TS1.bam`, step `compress` is executed to produce it. Then the second workflow is executed with target `TS1.bam.bai`, step `index` is executed with `TS1.bam` generated from the first run. In the last run, both steps are executed to generate `TS2.bam` and then `TS2.bam.bai`.

Simiarly, a step can provides specific file targets or multiple file targets through pattern matching. For example,

### Other targets

## Step option `shared`

Another common task of SoS steps is to provide some information through SoS variables. This can be achived by a `provides` option with `sos_variale` targets, but can be more easily implemented with a `shared` option.

In [3]:
# remove var in case it is defined already
%dict --del numNotebooks

# this step provides variable `numNotebooks`
[count: shared='numNotebooks']
import glob
numNotebooks = len(glob.glob('*.ipynb'))

[20]
depends: sos_variable('numNotebooks')
print("There are ${numNotebooks} notebooks in this directory")

There are 4 in this directory


As you can see, step prepare is called before step align_20 because align_20 depends on a sos_variable named var.

### Variable targets

## Executing workflows with auxiliary steps

You can execute forward-style workflows by specifying workflow name (can be `default`) from command line. The workflow can trigger auxiliary steps for the generation of unavailable targets. The workflows are executed with a mind-setting of **apply workflow to certain input file**.

You can execute a makefile-style workflow by specifying one or more targets using option `-t` (target). SoS would collect all auxiliary steps in the script and create DAGs to generate these targets. The workflows are executed with a mind-setting of **execute necessary steps to generate specified output files**. Forward-style workflows defined in the script, if defined, would be ignored.

You can specify both a forward-style workflow and one or more targets using the `-t` option. In this case SoS would create a DAG with both the forward-style workflow and steps to produce the specified targets. The DAG would then be trimmed to a sub-DAG that produce only specified targets. The usage is usually used to **produce only selected targets from a forward-style workflow**. In contrast to the second case where targets have to be targets of auxiliary steps, targets specified in this case can be any output targets from the forward-style workflow.

Let us use a slightly complex example with both forward-style and auxiliary steps to demonstrate these three cases.

In [4]:
%sandbox

[gzip: provides='{name}.gz']
input: "${name}"
print("> ${step_name} ${input} to ${output}")
run:
    touch ${output}

[download: provides='{name}.pdf']
print("> ${step_name} ${output}")
run:
    touch ${output}

[process_10]
print("> Running step ${step_name}")

[process_20]
depends: "step20.pdf"
output: "step20.out"
print("> Running step ${step_name} to produce ${output}")
run:
    touch ${output}

[process_30]
output: "step30.out"
print("> Running step ${step_name} to produce ${output}")
run:
    touch ${output}

[default]
print("Forward-style workflow")
sos_run("process")

print("\nMakefile-style workflow")
sos_run(targets="ms.pdf.gz")

print("\nTargets of forward-style workflow")
sos_run("process", targets="step20.out")

print("\nTargets from forward-style and makefile-style workflows")
sos_run("process", targets=["step20.out", "ms1.pdf.gz"])

os.remove('step20.out')
print("\nInvalid target step20.out")
sos_run(targets=["step20.out"])


Forward-style workflow
> download step20.pdf
> Running step process_10
> Running step process_20 to produce step20.out
> Running step process_30 to produce step30.out

Makefile-style workflow
> download ms.pdf
> gzip ms.pdf to ms.pdf.gz

Targets of forward-style workflow
> Running step process_10
> Running step process_20 to produce step20.out

Targets from forward-style and makefile-style workflows
> Running step process_10
> Running step process_20 to produce step20.out
> download ms1.pdf
> gzip ms1.pdf to ms1.pdf.gz

Invalid target step20.out


Failed to process statement print("Forward-style workflow"...p20.out"]): No step to generate target step20.out

This example has a forward-style workflow `process` in which step `process_20` depends on an auxiliary step `download`.

1. In the first case with command line equivalence

  ```bash
  sos run myscript process
  ```
  the forward-style workflow `process` is executed.

2. In the second example
  ```bash
  sos run myscript -t ms.pdf.gz
  ```
  two auxiliary steps `download` and `gzip` are called to produce target `ms.pdf.gz`.
  
3. In the third example
  ```bash
  sos run myscript process -t step20.out
  ```
  the `process` workflow is executed partially until it generates target `step20.out`.
  
4. In the fourth example
  ```bash
  sos run myscript process -t step20.out ms1.pdf.gz
  ```
  the `process` workflow is executed partially to produce target `step20.out`, and two auxiliary steps are executed to produce the additional target `ms1.pdf.gz`.
  
5. In the last example
  ```bash
  sos run myscript -t step20.out
  ```
  SoS could not find an auxiliary step to produce target `step20.out` and exited with error. Note that SoS would not try to execute a default workflow (workflow `default` or the only forwar-style workflow defined in the script) with the presence of option `-t`.